In [1]:
# Libraries to load data
from pyogrio import read_dataframe # faster than geopandas
import geopandas
import time
import pickle

# Import funcs script (make sure to refer to the right relative path)
from funcs import *

# Import other libraries
import os
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib.patheffects as PathEffects
from IPython.display import HTML
import  matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from IPython.display import HTML
from cartopy.io.img_tiles import GoogleTiles
import cartopy.io.img_tiles as cimgt
import matplotlib.image as mpimg
import random
import tqdm
import sys

### Load data

In [ ]:
# read shapefile
data = read_dataframe("../v_tracking_raw.shp")
data

In [3]:
# windfarm data (optional)
windfarms = read_dataframe("../windfarms.shp")
windfarms = windfarms.to_crs(3857)

windfarm_points = read_dataframe("../rivm_20230101_windturbines_2022_ashoogte.shp")
windfarm_points = windfarm_points.to_crs(3857)

PAWP_OWEZ_LUD_points = read_dataframe("../PAWP_OWEZ_LUD.shp")
PAWP_OWEZ_LUD_points = PAWP_OWEZ_LUD_points.to_crs(3857)

#### Load data of one bird or multiple
Depends on how your dataset looks like, in my case birds have an ID named 'ringnr' --> change functions for appropriate columns if necessary

In [3]:
# Around Africa: 'B-J14'
# In windfarms: 'B-J22'

# Single logger
single_gdf = subset_ringnr(data, ringnr = 'B-J14')
# reduced_single_gdf = reduce_by_movement(single_gdf) # reduce by movement 
reduced_single_gdf = reduce_by_time(single_gdf, resample='6h') # reduce by time
reduced_single_gdf = reduce_by_time(reduce_by_movement(single_gdf, significance=0.75), resample='6h') # reduce by time and position

single_windfarm_gdf = subset_ringnr(data, ringnr = 'W-J44')
reduced_single_windfarm_gdf = reduce_by_movement(single_windfarm_gdf)

# Multiple loggers
BH75 = reduce_by_time(subset_ringnr(data, ringnr = 'B-H75'), resample='6h')
BH92 = reduce_by_time(subset_ringnr(data, ringnr = 'B-H92'), resample='6h')
WJ62 = reduce_by_time(subset_ringnr(data, ringnr = 'W-J62'), resample='6h')
reduced_multiple_gdf = pd.concat([
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-H77'), resample='6h'), # and reduce
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-J14'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-J18'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-J22'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-J24'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-J26'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-J27'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'B-J32'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'W-J44'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'W-J59'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'W-J63'), resample='6h'),
                          reduce_by_time(subset_ringnr(data, ringnr = 'W-J72'), resample='6h'),
                          BH75[~((BH75['date_time'] >= '2021-01-01') & (BH75['date_time'] < '2022-01-01'))],
                          BH92[(BH92['date_time'] < '2022-07-01')],
                          WJ62[(WJ62['date_time'] < '2022-07-01')]])


In [4]:
# Only between time frame
# data_summer2021 = data[~((data['date_time'] >= '2021-06-01') & (data['date_time'] < '2021-09-01'))]

# All loggers
ringnumbers = data['ringnr'].unique().tolist()
all_reduce_list = []
for ringnr in ringnumbers:
    all_reduce_list.append(reduce_by_time(subset_ringnr(data, ringnr = ringnr), resample='6min'))

reduced_all_gdf = pd.concat(all_reduce_list)
reduced_all_gdf = reduced_all_gdf[(reduced_all_gdf['date_time'] >= '2021-05-15') & (reduced_all_gdf['date_time'] < '2021-07-01') & 
    (reduced_all_gdf['soort'] == 'Grote stern')]

### Animate

In [ ]:
# Plot single
plot_animation_single(reduced_single_gdf, frames_between_points = 10, interval = 15)

In [ ]:
# Plot multiple
plot_animation_multiple(reduced_multiple_gdf)

In [ ]:
plot_animation_multiple_stationary(reduced_all_gdf, zoom_level = 10, extent_margin = 0.45, lon_centre = 4.5, lat_centre = 52.75, with_tail = False,
                                    plot_windfarms = True, windfarms1 = windfarms, windfarms2 = PAWP_OWEZ_LUD_points)

In [ ]:
plot_animation_multiple(reduced_all_gdf)

In [ ]:
# Plot single on moving globe basemap
plot_animation_moving_globe(reduced_single_gdf, extent_margin = 35, frames_between_points = 20, interval = 15, zoom_level = 3, plot_windfarms=False, windfarms=None)

In [ ]:
# Plot single on moving basemap
plot_animation_moving(reduced_single_gdf, extent_margin = 4, frames_between_points = 10, interval = 15, zoom_level = 7, plot_windfarms=False, windfarms=None)